In [39]:
import numpy as np
from scipy.signal import butter
from scipy.signal import lfilter
import pandas as pd
import datetime as dt
import csv
from numpy import dot
import threading
import time
import os
import json
from dateutil.parser import parse
import warnings
# import TimeCheck as tc
import ClassificationN as clf
secret = [4,2,3,1]
cnt = 0
raw_time=[]
raw_value=[]
raw=[]
peek_time = []
peek_index = []
json_data = []
isLayer1 = False
isLayer2 = False

In [43]:
def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band', analog=False)
    return b, a


def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = lfilter(b, a, data)
    return y
#mave time을 datetime형식으로
def time_transe(time):
    if time[0:2] == '오전':
        time = time[3:]
        if time[3:5] == '12':
            temp = '00'+time[5:]
            t_date = parse(temp)
        else:
            t_date = parse(time)
    else:
#         h = int(time[3:4])+12
        h = int(time[3:5])+12
#         h = int(time[3:5])
#         time = time[4:]
        time = time[5:]
        t_date = parse(str(h)+time)
#     print(t_date)
    return t_date

#Mave EEG를 실시간으로 받아오기
def loadRawdata(path):
    print('raw thread')
    global raw_time
    global raw_value
    global raw
    try:
        #     path = 'C:/MAVE_RawData/2021-06-05_오후 5_18/Rawdata.txt'
        # path2 = 'C:/MAVE_RawData/오재무/2021-06-04_오후 5_52/Rawdata.txt'
    #     print(path)
        temp = pd.read_csv(path,delimiter='\t',encoding='cp949')
        # temp2 = pd.read_csv(path2,delimiter='\t',encoding='cp949')
        t_fp1 = temp['Time']
        e_fp1 = temp['EEG_Fp1']
        # t_fp2 = temp2['Time']
        # e_fp2 = temp2['EEG_Fp1']
        
        raw_time=t_fp1
        raw_value=e_fp1
        raw=temp
        
    except TypeError:
        pass
#     threading.Timer(1,loadRawdata,args = (path)).start()

#패널과 연동
def load_json():
    print('json thread')
    global peek_time
    global peek_index
    global json_data
    global isLayer1
    global isLayer2
    with open('C:\\Users\\toddl\\OneDrive\\바탕 화면\\ERP\\Project_ERP_Data\\timeData.json', 'r') as f:
        json_data = json.load(f)
    peek_time = json_data['peekTime']
    peek_index = json_data['peekIndex']
    isLayer1 = json_data['isLayer1']
    isLayer2 = json_data['isLayer2']
#우형 만듬
def load_Json():
    print("Debug7")
    with open('C:\\Users\\toddl\\OneDrive\\바탕 화면\\ERP\\Project_ERP_Data\\timeData.json', 'r') as f:
        json_data = json.load(f)
    return json_data

def write_json(num,sel,pas = 3):
    json_data = load_Json()
    if sel == 1:
        json_data['isLayer1'] = num
    elif sel == 2:
        json_data['isLayer2'] = num
    else :
        json_data['password'][num] = pas
    with open('C:\\Users\\toddl\\OneDrive\\바탕 화면\\ERP\\Project_ERP_Data\\timeData.json', 'w') as f:
        json.dump(json_data, f)
        #json_data = json.load(f)  
    print("write_json")
  
    
    
    
def creat_Path(now):
    now = now
    t_month = ""
    t_day = ""
    t_minute = ""

    if now.month<10:
        t_month='0'+str(now.month)

    if now.day<10:
        t_day='0'+str(now.day)
    else:
        t_day=str(now.day)
        
    if now.minute < 10:
        t_minute = '0'+str(now.minute)
    else:
        t_minute = str(now.minute)

    t_hm = ""

    if now.hour>12 and now.hour <= 23:
        t_hm="오후 "+str(now.hour-12)+'_'+str(t_minute)
    elif now.hour == 12:
        t_hm="오후 "+str(now.hour)+'_'+str(t_minute)
    elif now.hour == 0:
         t_hm="오전 "+str(now.hour+12)+'_'+str(t_minute)
    else:
        t_hm="오전 "+str(now.hour)+'_'+str(t_minute)

    t_date = str(now.year)+'-'+str(t_month)+'-'+str(t_day)+'_'+t_hm
    
#     print(t_date)
    return t_date

#패널과 mave 시간을 같은 형식으로
def jsonMave(raw_time,peek_time):
    time_str = dt.datetime.now()
    start_index = 0
    start_time = dt.datetime.now()
    break_point = False
    n200_start=''
    n200_end=''
    p300_start=''
    p300_end=''
    i=0
    for p in reversed(peek_time):
        if break_point == True:
            print('break point!')
            break
        if p=='':
#             print('p is null')
            i+=1
            continue
        elif p!='' :
            start_time = parse(p[11:-3])
            for v in raw_time:
                time_str = time_transe(v)
            
                if start_time >= time_str and start_time - time_str<=dt.timedelta(milliseconds = 100):
                    print('suc')
                    n200_start = start_time+dt.timedelta(milliseconds = 150)
                    n200_end = start_time+dt.timedelta(milliseconds = 250)
                    p300_start = start_time+dt.timedelta(milliseconds = 250)
                    p300_end = start_time+dt.timedelta(milliseconds = 350)
                    break_point = True
                    break
    return n200_start, n200_end, p300_start, p300_end, i
               
#n200과 p300시간대를 추정            
def detection(mave_time_list,n200_start, n200_end, p300_start, p300_end):
    p_start_i=-1
    p_end_i=-1
    n_start_i=-1
    n_end_i=-1
    for idx, t in mave_time_list.items(): 
        if (n200_start - time_transe(t) <= dt.timedelta(milliseconds = 100)):
            if n_start_i == -1:
                n_start_i = idx
                print('nplist_0 : {}'.format(idx))
        if (n200_end - time_transe(t) <= dt.timedelta(milliseconds = 100)):
            if n_end_i == -1:
                n_end_i = idx
                print('nplist_1 : {}'.format(idx))
        if (p300_start - time_transe(t) <= dt.timedelta(milliseconds = 100)):
             if p_start_i == -1:
                p_start_i = idx
        if (p300_end - time_transe(t) <= dt.timedelta(milliseconds = 100)):
            if p_end_i==-1:
                p_end_i = idx
   #위 조건들을 만족하지 못하는 경우를 위해
    if n_start_i == -1:
        n_start_i = 40
    n_end_i=n_start_i+35
    p_start_i=n_end_i+1
    p_end_i = p_start_i + 30
    np_list = [n_start_i,n_end_i,p_start_i,p_end_i]
    return np_list

In [37]:
#현재 시간으로 mave rawdata접근
now = dt.datetime.now()

def main():
    global raw_time
    global raw_value
    global raw
    global peek_time
    global peek_index
    global json_data
    global isLayer1
    global isLayer2
    global cnt
    no_empty_json = []
    warnings.filterwarnings('ignore') 
    t_date = creat_Path(now)
    path = 'C:/MAVE_RawData/'+t_date+'/Rawdata.txt'
    
    while(True) :
    #MAVE Rawdata 계속해서 로드
        loadRawdata(path)
        threading.Timer(1,loadRawdata,args = (path)).start()
        
    #JSON 계속해서 로드
        p_idx=-1
        load_json()
        threading.Timer(1,load_json).start()
        n200_start=0
        n200_end=0
        p300_start=0
        p300_end = 0
    
    #Peek time (json) 과 MAVE Rawdata를 비교
        if len(peek_time) == 0:
            print('len 0')
        print('peek_index : ',peek_index[0:10])
        n200_start, n200_end, p300_start, p300_end, p_idx = jsonMave(raw_time, peek_time)
        print('pre idx :', p_idx)
        p_idx = len(peek_index)- 1 - p_idx
        print('init n200,p300 value')
        print('idx : ', p_idx)
        print('peek_index :',peek_index[p_idx])
        
        #Rawdata filtering
        Fs = 512
        lowcut = 0.5 #0.5
        highcut = 30 #30
        m_lpf = butter_bandpass_filter(raw_value,lowcut, highcut, Fs, order=5)
        print('m_lpf len : {}'.format(len(m_lpf)), end='\n')
        
        #상
        if peek_index[p_idx] == 1:
            print('최종 peek index : 1')
            np_list = detection(raw_time,n200_start, n200_end, p300_start, p300_end)
            
            if np_list[0]==40:
                print('<<predict false>>')
            else:
                try:
                    n200 = m_lpf[np_list[0]:np_list[1]+1]
                    p300 = m_lpf[np_list[2]:np_list[3]]

                    print('np_list입니다 : {}'.format(np_list))
#                     print('n200 , p300 : {}, {}'.format(n200, p300))

                    # Classification을 구현, 사전에 데이터로 W,b를 정해줌
                    W = [6.04723296e-04, 9.13397135e-01]
                    b = 0.21733827
                    
                    # n200의 min값과 p300의 max값으로 Classification.predict함수에 넣어줌
                    if clf.predict([min(n200), max(p300)],W,b) == 0:
                        print('\n')
                        print('<<predict false>>')
                        print('\n')
                    else:
                        print('\n')
                        print('<<predict up>>')
                        print('\n')
                        if isLayer2=='False':
                            write_json(True,2)
                        else:
                            write_json(cnt,3,pas = 1)
                            write_json(True,1)
                            cnt+=1
                        
                except:
                    print('\n')
                    print('<<predict false>>')
                    print('\n')
        #하
        elif peek_index[p_idx] == 2:
            print('최종 peek index : 2')
            np_list = detection(raw_time,n200_start, n200_end, p300_start, p300_end)
            
            if np_list[0]==40:
                print('<<predict false>>')
            else:
                try:
                    n200 = m_lpf[np_list[0]:np_list[1]+1]
                    p300 = m_lpf[np_list[2]:np_list[3]]

                    print('np_list입니다 : {}'.format(np_list))
#                     print('n200 , p300 : {}, {}'.format(n200, p300))

                    # Classification을 구현, 사전에 데이터로 W,b를 정해줌
                    W = [6.04723296e-04, 9.13397135e-01]
                    b = 0.21733827
                    
                    # n200의 min값과 p300의 max값으로 Classification.predict함수에 넣어줌
                    if clf.predict([min(n200), max(p300)],W,b) == 1:
                        print('\n')
                        print('<<predict false>>')
                        print('\n')
                    else:
                        print('\n')
                        print('<<predict up>>')
                        print('\n')
                        if isLayer2=='False':
                            write_json(True,2)
                        else:
                            write_json(cnt,3,pas = 1)
                            write_json(True,1)
                            cnt+=1
                        
                except:
                    print('\n')
                    print('<<predict false>>')
                    print('\n')

In [38]:
main()

raw thread
json thread
peek_index :  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
pre idx : 100
init n200,p300 value
idx :  -1
peek_index : 0
m_lpf len : 168
raw thread
json thread
peek_index :  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
pre idx : 100
init n200,p300 value
idx :  -1
peek_index : 0
m_lpf len : 172
raw thread
json thread
peek_index :  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
pre idx : 100
init n200,p300 value
idx :  -1
peek_index : 0
m_lpf len : 174
raw thread
json thread
peek_index :  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
pre idx : 100
init n200,p300 value
idx :  -1
peek_index : 0
m_lpf len : 174
raw thread
json thread
peek_index :  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
pre idx : 100
init n200,p300 value
idx :  -1
peek_index : 0
m_lpf len : 178
raw thread
json thread
peek_index :  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
pre idx : 100
init n200,p300 value
idx :  -1
peek_index : 0
m_lpf len : 184
raw thread
json thread
peek_index :  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
pre idx : 100
init n200,p300 value
idx :  -1
peek_index : 0
m_lpf le

Exception in thread Thread-2488:
Traceback (most recent call last):
  File "C:\Users\toddl\anaconda3\lib\threading.py", line 932, in _bootstrap_inner
    self.run()
  File "C:\Users\toddl\anaconda3\lib\threading.py", line 1254, in run
Exception in thread Thread-2490:
Traceback (most recent call last):
  File "C:\Users\toddl\anaconda3\lib\threading.py", line 932, in _bootstrap_inner
Exception in thread Thread-2492:
    Traceback (most recent call last):
self.run()
      File "C:\Users\toddl\anaconda3\lib\threading.py", line 1254, in run
self.function(*self.args, **self.kwargs)  File "C:\Users\toddl\anaconda3\lib\threading.py", line 932, in _bootstrap_inner

TypeError: loadRawdata() takes 1 positional argument but 46 were given
        self.function(*self.args, **self.kwargs)
TypeError: loadRawdata() takes 1 positional argument but 46 were given
self.run()
  File "C:\Users\toddl\anaconda3\lib\threading.py", line 1254, in run
    Exception in thread Thread-2494self.function(*self.args, **

pre idx : json thread100

init n200,p300 value
idx :  -1
peek_index : 0
json thread
m_lpf len : 212
raw thread
json thread
json thread
json thread
json thread
json thread
json thread
json thread
peek_index :  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
pre idx : 100
init n200,p300 value
idx :  -1
peek_index : 0


    self.function(*self.args, **self.kwargs)
TypeErrorException in thread     : Exception in thread Thread-2510self.function(*self.args, **self.kwargs)    
TypeError: loadRawdata() takes 1 positional argument but 46 were given
loadRawdata() takes 1 positional argument but 46 were givenself.function(*self.args, **self.kwargs)    :
Thread-2508Exception in thread :
self.function(*self.args, **self.kwargs)
Thread-2512:
Traceback (most recent call last):
  File "C:\Users\toddl\anaconda3\lib\threading.py", line 932, in _bootstrap_inner
Traceback (most recent call last):

TypeErrorTraceback (most recent call last):
  File "C:\Users\toddl\anaconda3\lib\threading.py", line 932, in _bootstrap_inner

TypeError  File "C:\Users\toddl\anaconda3\lib\threading.py", line 932, in _bootstrap_inner
: : loadRawdata() takes 1 positional argument but 46 were givenException in thread 
    Thread-2514    self.run()self.run()
loadRawdata() takes 1 positional argument but 46 were given
  File "C:\Users\toddl\ana

json thread
json threadjson thread
json thread

json thread
m_lpf len : 220
raw thread
json threadjson thread

json thread
json thread
json thread
peek_index :  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
pre idx : 100
init n200,p300 value
idx :  -1
peek_index : 0
json thread
m_lpf len : 234
raw thread
json thread
json thread
json thread
json thread
peek_index :  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
pre idx : 100
init n200,p300 value
idx :  -1
peek_index : 0


...

nplist_0 : 644
nplist_1 : 654
np_list입니다 : [644, 679, 680, 710]
n200 , p300 : [-1.56642797e-05 -1.55610682e-05 -1.57279622e-05 -1.60727409e-05
 -1.64600643e-05 -1.67178549e-05 -1.66731012e-05 -1.61938981e-05
 -1.52080514e-05 -1.37039384e-05 -1.17274678e-05 -9.37232640e-06
 -6.75511394e-06 -3.98423481e-06 -1.15018440e-06  1.66019737e-06
  4.35919514e-06  6.87426640e-06  9.15527596e-06  1.11579571e-05], []


<<predict up>>


raw thread
json thread
peek_index :  [1, 1, 1, 1, 2, 1, 2, 1, 1, 1]
json thread


Exception in thread Thread-2632:
Traceback (most recent call last):
  File "C:\Users\toddl\anaconda3\lib\threading.py", line 932, in _bootstrap_inner
    self.run()
  File "C:\Users\toddl\anaconda3\lib\threading.py", line 1254, in run
    self.function(*self.args, **self.kwargs)
TypeError: loadRawdata() takes 1 positional argument but 46 were given


suc
break point!
pre idx : 90
init n200,p300 value
idx :  9
peek_index : 1
m_lpf len : 772
최종 peek index : 1
nplist_0 : 710
nplist_1 : 712
np_list입니다 : [710, 745, 746, 776]
n200 , p300 : [-7.42141123e-06 -7.95209048e-06 -8.03066553e-06 -7.69715044e-06
 -7.04924286e-06 -6.21130511e-06 -5.29999376e-06 -4.40885678e-06
 -3.60989725e-06 -2.94734290e-06 -2.42099095e-06 -1.98781587e-06
 -1.59527448e-06 -1.22073609e-06 -8.87827854e-07 -6.59702214e-07
 -6.23454037e-07 -8.65959776e-07 -1.43678629e-06 -2.31787864e-06
 -3.43165856e-06 -4.68707625e-06 -6.02291672e-06 -7.41297772e-06
 -8.84359924e-06 -1.02957201e-05 -1.17353386e-05 -1.30996765e-05
 -1.42917126e-05 -1.52006519e-05 -1.57310546e-05 -1.58173468e-05
 -1.54315251e-05 -1.45915908e-05 -1.33537071e-05 -1.17888767e-05], [-9.97373384e-06 -7.99290402e-06 -5.91929217e-06 -3.78085708e-06
 -1.55698551e-06  7.84311517e-07  3.23579190e-06  5.72157065e-06
  8.10666741e-06  1.02291913e-05  1.19238022e-05  1.30391896e-05
  1.34770534e-05  1.32454263e-0

Exception in thread Thread-2634:
Traceback (most recent call last):
  File "C:\Users\toddl\anaconda3\lib\threading.py", line 932, in _bootstrap_inner
    self.run()
  File "C:\Users\toddl\anaconda3\lib\threading.py", line 1254, in run
    self.function(*self.args, **self.kwargs)
TypeError: loadRawdata() takes 1 positional argument but 46 were given


KeyboardInterrupt: 